<a href="https://colab.research.google.com/github/KhamessiTaha/CarVisionProject/blob/main/Car_Model_Classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install rarfile
from google.colab import drive
import os
import pandas as pd
import rarfile


In [ ]:

# Mount Google Drive
drive.mount('/content/drive')




Mounted at /content/drive


In [ ]:
# Path to the dataset folder
dataset_path = '/content/drive/My Drive/dataset'

# Extract RAR archives if not already extracted
def extract_rar(archive_path, destination_path):
    if not os.path.exists(destination_path):
        os.makedirs(destination_path)
    with rarfile.RarFile(archive_path, 'r') as rar_ref:
        rar_ref.extractall(destination_path)

# Check if train, test, and validation folders already exist
train_extracted = os.path.exists(os.path.join(dataset_path, 'train'))
test_extracted = os.path.exists(os.path.join(dataset_path, 'test'))
val_extracted = os.path.exists(os.path.join(dataset_path, 'val'))

# Extract train, test, and validation archives if not already extracted
if not train_extracted:
    extract_rar(os.path.join(dataset_path, 'train.rar'), dataset_path)
if not test_extracted:
    extract_rar(os.path.join(dataset_path, 'test.rar'), dataset_path)
if not val_extracted:
    extract_rar(os.path.join(dataset_path, 'val.rar'), dataset_path)

# Load CSV files
train_labels = pd.read_csv(os.path.join(dataset_path, 'train_labels.csv'))
test_labels = pd.read_csv(os.path.join(dataset_path, 'test_labels.csv'))
val_labels = pd.read_csv(os.path.join(dataset_path, 'val_labels.csv'))

# Display the first few rows of the train_labels dataframe
print("Train Labels:")
print(train_labels.head())

# Display the first few rows of the test_labels dataframe
print("\nTest Labels:")
print(test_labels.head())

# Display the first few rows of the val_labels dataframe
print("\nValidation Labels:")
print(val_labels.head())

Train Labels:
        Image Make     Model  Year
0  657068.jpg  Kia  Sportage  2015
1  657050.jpg  Kia  Sportage  2015
2  656981.jpg  Kia  Sportage  2015
3  657192.jpg  Kia  Sportage  2015
4  657146.jpg  Kia  Sportage  2015

Test Labels:
        Image Make     Model  Year
0  657235.jpg  Kia  Sportage  2015
1  657010.jpg  Kia  Sportage  2015
2  657172.jpg  Kia  Sportage  2015
3  657229.jpg  Kia  Sportage  2015
4  657194.jpg  Kia  Sportage  2015

Validation Labels:
        Image Make     Model  Year
0  656975.jpg  Kia  Sportage  2015
1  657121.jpg  Kia  Sportage  2015
2  657110.jpg  Kia  Sportage  2015
3  657242.jpg  Kia  Sportage  2015
4  657009.jpg  Kia  Sportage  2015


In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import numpy as np

In [ ]:
# Define image dimensions
img_width, img_height = 300, 300

# Define batch size
batch_size = 32

In [ ]:
# ImageDataGenerator for data augmentation
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True
)


In [ ]:
# ImageDataGenerator for validation and testing data (no augmentation)
test_datagen = ImageDataGenerator(rescale=1./255)

In [ ]:


# Convert lists to NumPy arrays if necessary
train_labels_array = np.asarray(train_labels)
train_directory_array = np.asarray(os.path.join(dataset_path, 'train'))

# Load and preprocess training data with NumPy arrays
train_generator = train_datagen.flow_from_dataframe(
    dataframe=train_labels_array,
    directory=train_directory_array,
    x_col='Image',
    y_col=['Make', 'Model', 'Year'],
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='raw'
)


IndexError: only integers, slices (`:`), ellipsis (`...`), numpy.newaxis (`None`) and integer or boolean arrays are valid indices

In [ ]:
# Load and preprocess validation data
validation_generator = test_datagen.flow_from_dataframe(
    dataframe=val_labels,
    directory=os.path.join(dataset_path, 'val'),
    x_col='Image',
    y_col=['Make', 'Model', 'Year'],
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='raw'
)

Found 217757 validated image filenames.


In [ ]:
# Load and preprocess testing data
test_generator = test_datagen.flow_from_dataframe(
    dataframe=test_labels,
    directory=os.path.join(dataset_path, 'test'),
    x_col='Image',
    y_col=['Make', 'Model', 'Year'],
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='raw'
)

Found 224680 validated image filenames.


In [ ]:
from tensorflow.keras.applications import ResNet50V2
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D

In [ ]:
# Load pre-trained ResNet50V2 model
base_model = ResNet50V2(weights='imagenet', include_top=False, input_shape=(img_width, img_height, 3))

94668760/94668760 [==============================] - 1s 0us/step


In [ ]:
# Add new classification layers
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
predictions = Dense(3, activation='softmax')(x)

In [ ]:
# Define new model with modified architecture
model = Model(inputs=base_model.input, outputs=predictions)

In [ ]:
# Freeze base layers (optional)
for layer in base_model.layers:
    layer.trainable = False


In [ ]:
# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
# Display model summary
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, 300, 300, 3)]        0         []                            
                                                                                                  
 conv1_pad (ZeroPadding2D)   (None, 306, 306, 3)          0         ['input_1[0][0]']             
                                                                                                  
 conv1_conv (Conv2D)         (None, 150, 150, 64)         9472      ['conv1_pad[0][0]']           
                                                                                                  
 pool1_pad (ZeroPadding2D)   (None, 152, 152, 64)         0         ['conv1_conv[0][0]']          
                                                                                              

In [ ]:
epochs = 10


In [ ]:
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping

# Define callbacks
checkpoint = ModelCheckpoint('model_checkpoint.h5', monitor='val_loss', save_best_only=True, mode='min', verbose=1)
early_stopping = EarlyStopping(monitor='val_loss', patience=3, verbose=1)

# Train the model with callbacks
history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples / batch_size,  # Changed here
    epochs=epochs,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples / batch_size,  # Changed here
    callbacks=[checkpoint, early_stopping]
)


ValueError: Failed to convert a NumPy array to a Tensor (Unsupported object type int).

In [ ]:
[print(i.shape, i.dtype) for i in model.inputs]
[print(o.shape, o.dtype) for o in model.outputs]
[print(l.name, l.input_shape, l.dtype) for l in model.layers]

(None, 300, 300, 3) <dtype: 'float32'>
(None, 3) <dtype: 'float32'>
input_1 [(None, 300, 300, 3)] float32
conv1_pad (None, 300, 300, 3) float32
conv1_conv (None, 306, 306, 3) float32
pool1_pad (None, 150, 150, 64) float32
pool1_pool (None, 152, 152, 64) float32
conv2_block1_preact_bn (None, 75, 75, 64) float32
conv2_block1_preact_relu (None, 75, 75, 64) float32
conv2_block1_1_conv (None, 75, 75, 64) float32
conv2_block1_1_bn (None, 75, 75, 64) float32
conv2_block1_1_relu (None, 75, 75, 64) float32
conv2_block1_2_pad (None, 75, 75, 64) float32
conv2_block1_2_conv (None, 77, 77, 64) float32
conv2_block1_2_bn (None, 75, 75, 64) float32
conv2_block1_2_relu (None, 75, 75, 64) float32
conv2_block1_0_conv (None, 75, 75, 64) float32
conv2_block1_3_conv (None, 75, 75, 64) float32
conv2_block1_out [(None, 75, 75, 256), (None, 75, 75, 256)] float32
conv2_block2_preact_bn (None, 75, 75, 256) float32
conv2_block2_preact_relu (None, 75, 75, 256) float32
conv2_block2_1_conv (None, 75, 75, 256) float3

[None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,

#TRAIN THE MODEL


In [ ]:
# Define number of epochs
epochs = 10

# Train the model
history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // batch_size,
    epochs=epochs,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // batch_size
)


In [ ]:
# Evaluate the model on testing data
loss, accuracy = model.evaluate(test_generator, steps=test_generator.samples // batch_size)
print("Test Loss:", loss)
print("Test Accuracy:", accuracy)
